In [6]:
import time
from langserve import RemoteRunnable

# Get the remote runnable from the FastAPI server created in jockey.py
jockey = RemoteRunnable("http://localhost:8000/jockey")
# Guarantee unique session ID every time for testing
session_id = time.time()

In [16]:
def parse_langserve_events(event: dict):
    """Used to parse events emitted from Jockey when called as an API."""
    if event["event"] == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            print(f"{content}", end="", flush=True)
            # Attempts to maintain a consistent token rendering rate for a smoother UX
            time.sleep(0.05)
    elif event["event"] == "on_tool_start":
        tool = event["name"]
        print(f"Running => {event["data"]}", flush=True)
        print(f"Running => {tool}", flush=True)
    elif event["event"] == "on_tool_end":
        tool = event["name"]
        print(f"Finished running {tool}", flush=True)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (4203199828.py, line 11)

In [15]:
tool_names = ["video-search", "download-video", "combine-clips", "remove-segment"]
### 
# Used to limit the events by type streamed from the FastAPI server when Jockey is running
# This is reduces the amount of data transmitted to the client from the server and reduces the parsing logic required.
# Most events aren't relevant for rendering the client side UI but can be added as need.
###
include_types = ["chat_model"]
include_names=["AzureChatOpenAI"] + tool_names

#### 
# We make an async call to Jockey with the `astream_events` endpoint so that we can render events and tokens as they are emitted.
# This is equivalent to making a `POST http://localhost:8000/jockey/astream_events` call
###
async for event in jockey.astream_events({"input": "Use index id 659f2e829aba4f0b402f6488 to find the top clip of a touchdown"}, 
                                         {'configurable': { 'session_id': session_id }}, version="v1",
                                         include_types=include_types, include_names=include_names):
    parse_langserve_events(event)

Running => video-search
Finished running video-search
I have found the top clip of a touchdown using the index ID "659f2e829aba4f0b402f6488". Here are the details of the clip:

- **Score**: 90.97
- **Start Time**: 138 seconds
- **End Time**: 139 seconds
- **Video ID**: "659f39841cbcab2e74d8b308"
- **Confidence**: High
- **Thumbnail URL**: ![Touchdown Thumbnail](https://project-one-thumbnail.s3.us-west-2.amazonaws.com/659f39841cbcab2e74d8b308/139.jpeg)
- **Video URL**: [Touchdown Video Clip](https://deuqpmn4rs7j5.cloudfront.net/64014aa08e079b9c96f4a55c/659f397c9aba4f0b402f649d/stream/5ca23f17-bdb4-4dc6-bbee-dfc054fc9d74.m3u8)

Would you like to download this clip or perform any other actions with it?

```json
{
  "tools": [
    {
      "tool_name": "video-search",
      "tool_input": {
        "query": "touchdown",
        "index_id": "659f2e829aba4f0b402f6488",
        "top_n": 1,
        "group_by": "clip"
      },
      "tool_output": [
        {
          "score": 90.97,
          "